# Mocked Authorities

#### Use SDX stack for authorities temporarily

In [1]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
#export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
#export OS_REGION_NAME='CHI@TACC'
#export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom
export AUTH_INFO_FILE=${WORKING_DIR}/authority_info
export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info

export AUTH_NETWORK='sharednet1'
export AUTH_NODE_NAME=${USERNAME}-auth
export AUTH_STACK_NAME=${USERNAME}-auth



#RIAK Info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo RIAK_PUBLIC_IP: $RIAK_PUBLIC_IP

export OS_REGION_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_REGION_NAME | cut -d " " -f 2`
export OS_PROJECT_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_PROJECT_NAME | cut -d " " -f 2`
echo OS_REGION_NAME $OS_REGION_NAME
echo OS_PROJECT_NAME $OS_PROJECT_NAME


RIAK_PUBLIC_IP: 192.5.87.143
OS_REGION_NAME CHI@UC
OS_PROJECT_NAME CH-816532


In [2]:
#Clear Authoriy Info
echo "" > $AUTH_INFO_FILE
echo AUTH_NODE_NAME $AUTH_NODE_NAME >> $AUTH_INFO_FILE
echo AUTH_STACK_NAME $AUTH_STACK_NAME >> $AUTH_INFO_FILE

#### Create Lease (if necessary)

In [3]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation

# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-auth-lease"
echo LEASE_NAME $LEASE_NAME >> $AUTH_INFO_FILE

NODE_MIN=1
NODE_MAX=1

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

pruth-cici-universal-lease2
LEASE_NAME: pruth-cici-universal-lease2

LEASE_NAME pruth-cici-universal-lease2
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RIAK_NODE_NAME pruth-riak
RIAK_STACK_NAME pruth-riak
RIAK_PUBLIC_IP 192.5.87.143
LEASE_NAME pruth-cici-universal-lease2


In [4]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

#Poll for node to be ready for ssh
while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_STATUS ${RESERVATION_STATUS}
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $AUTH_INFO_FILE

cat $AUTH_INFO_FILE



RESERVATION_ID 480a3fde-d50b-40d8-b48b-781729342b20
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RESERVATION_STATUS ACTIVE
RESERVATION_STATUS ACTIVE

AUTH_NODE_NAME pruth-auth
AUTH_STACK_NAME pruth-auth
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc


#### Start Authority SAFE Server Stack

In [5]:
RESERVATION_RESOURCE_ID=`cat $AUTH_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
AUTH_NODE_NAME=`cat $AUTH_INFO_FILE | grep AUTH_NODE_NAME | cut -d " " -f 2`
AUTH_STACK_NAME=`cat $AUTH_INFO_FILE | grep AUTH_STACK_NAME | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
#IMAGE='CC-CentOS7'
IMAGE='CICI-SAFE.v1'
echo  RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image $IMAGE \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${RESERVATION_RESOURCE_ID} \
                        --security-group default  \
                        --nic net-id=${AUTH_NETWORK} \
                        --wait \
                        ${AUTH_NODE_NAME}

echo Server creation complete! 
echo ${AUTH_NODE_NAME} is `openstack server show --format value -c status ${AUTH_NODE_NAME}`


RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
Creating servers... This will take several minutes!

+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         | nova                                   |
| OS-EXT-SRV-ATTR:host                | admin01-ironic                         |
| OS-EXT-SRV-ATTR:hypervisor_hostname | ec7da18b-ed66-48c7-951d-d9858a786a0a   |
| OS-EXT-SRV-ATTR:instance_name       | instance-0000836a                      |
| OS-EXT-STS:power_state              | Running                                |
| OS-EXT-STS:task_state               | None                                   |
| OS-EXT-STS:vm_state                 | active                              

In [6]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   echo Getting status for $AUTH_NODE_NAME
   server_status=$(openstack server list --format value -c Status --name "$AUTH_NODE_NAME")
   echo status $server_status
done

# Request a public floating IP (in the 'public' network)
AUTH_PUBLIC_IP=$(openstack floating ip create public --format value -c floating_ip_address)

echo AUTH_PUBLIC_IP $AUTH_PUBLIC_IP

# Assign a public floating IP to $USER-first-server
openstack server add floating ip "$AUTH_NODE_NAME" "$AUTH_PUBLIC_IP"
echo AUTH_PUBLIC_IP $AUTH_PUBLIC_IP >> $AUTH_INFO_FILE

Getting status for pruth-auth
status ACTIVE
AUTH_PUBLIC_IP 192.5.87.89


In [7]:
RESERVATION_RESOURCE_ID=`cat $AUTH_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
AUTH_NODE_NAME=`cat $AUTH_INFO_FILE | grep AUTH_NODE_NAME | cut -d " " -f 2`
AUTH_STACK_NAME=`cat $AUTH_INFO_FILE | grep AUTH_STACK_NAME | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

#echo Creating Authority SAFE server. This will take several minutes!
#openstack stack create  --max-width 80 \
#        --template ${WORKING_DIR}/SDX/safe_server.yaml \
#        --parameter "key_name=${SSH_KEY_NAME}" \
#        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
#        --parameter "network_name=${AUTH_NETWORK}" \
#        --parameter "safe_node_name=${AUTH_NODE_NAME}" \
#        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
#        --wait \
#        ${AUTH_STACK_NAME}



In [8]:
#echo Authority server creation complete! 
#AUTH_NODE_NAME=`cat $AUTH_INFO_FILE | grep AUTH_NODE_NAME | cut -d " " -f 2`
#
#AUTH_PUBLIC_IP=`openstack server show  --format value -c addresses ${AUTH_NODE_NAME} | cut -d " " -f 2`
#echo The Authority public IP is $AUTH_PUBLIC_IP
#echo AUTH_PUBLIC_IP $AUTH_PUBLIC_IP >> $AUTH_INFO_FILE

In [9]:
#AUTH_PUBLIC_IP=`openstack server show --format value -c addresses ${AUTH_NODE_NAME} | cut -d " " -f 2`
AUTH_PUBLIC_IP=`cat $AUTH_INFO_FILE | grep AUTH_PUBLIC_IP | cut -d " " -f 2`
echo AUTH_PUBLIC_IP is $AUTH_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${AUTH_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo Authority node ready for ssh

AUTH_PUBLIC_IP is 192.5.87.89
Authority node ready for ssh


In [10]:
AUTH_PUBLIC_IP=`cat $AUTH_INFO_FILE | grep AUTH_PUBLIC_IP | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

REMOTE_SCRIPT=/home/cc/script.sh
LOCAL_SCRIPT=${WORKING_DIR}/riak_script.sh

cat << EOF > $LOCAL_SCRIPT
#!/bin/bash
sudo sh -c 'echo "PermitRootLogin yes" >>/etc/ssh/sshd_config'
#sudo yum install -y docker vim mlocate git maven
sudo systemctl start docker

echo Start SAFE Server in Container
#sudo docker pull yaoyj11/safeserver-v7
sudo docker run -i -t -d -p 7777:7777 -h safe --name safe yaoyj11/safeserver-v7
sudo docker exec -d safe /bin/bash -c  "cd /root/safe;sed -i 's/RIAKSERVER/$RIAK_PUBLIC_IP/g' safe-server/src/main/resources/application.conf;./prdn.sh"
           
echo Create ssh key
sudo sh -c 'ssh-keygen -t rsa -b 4096  -P "" -f "/root/.ssh/id_rsa"  -q'
sudo sh -c 'cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys2'
sudo sh -c 'chmod 600 ~/.ssh/authorized_keys2'

echo Boot Script Done!
EOF

echo Copying script to cc@$AUTH_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SCRIPT \
    cc@${AUTH_PUBLIC_IP}:${REMOTE_SCRIPT}
echo Done

echo Running script
ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${AUTH_PUBLIC_IP} "chmod +x "$REMOTE_SCRIPT" && "$REMOTE_SCRIPT

echo Done

Copying script to cc@192.5.87.89
Done
Running script
Start SAFE Server in Container
30dcec31de63f53c346af1db59217617a245639a47cc8559d4e02621525ea267
Create ssh key
Boot Script Done!
Done


#### Make delegations to a user
##### Geni delegations, IP delegation, Tag delegation

#### (STEP 3a)

In [ ]:
USERKEYHASH="2JAOVClQyaajkrpe7kxTtqbzW3x__AmwgFfKXyo63Q8="
USERSLICE=${USERNAME}-test
USERIP="192.168.42.1/24"
USERTAG="tag0"

REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

#### (STEP 3b) -- Then go back to "client.ipynb"

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${AUTH_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock auth ${USERKEYHASH} ${USERSLICE} ${USERIP} ${USERTAG} ${AUTH_PUBLIC_IP}"
    
echo Authority delegations done

#### Clean Up Authority

In [ ]:
AUTH_STACK_NAME=`cat $AUTH_INFO_FILE | grep AUTH_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $AUTH_STACK_NAME

LEASE_NAME=`cat $AUTH_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $AUTH_INFO_FILE
mv $AUTH_INFO_FILE ${AUTH_INFO_FILE}.old